In [138]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import accuracy_score

X_train=torch.tensor(pd.read_csv('data/X_train_dec.csv').values, dtype=torch.float32).unsqueeze(1).transpose(0,1)
X_test=torch.tensor(pd.read_csv('data/X_test_dec.csv').values,dtype=torch.float32).unsqueeze(1).transpose(0,1)
y_train=torch.tensor(pd.read_csv('data/y_train_dec.csv').values,dtype=torch.float32).unsqueeze(0)
y_test=pd.read_csv('data/y_test_dec.csv').to_numpy()

In [139]:
X_train.shape,y_train.shape

(torch.Size([1, 6519, 32]), torch.Size([1, 6519, 1]))

In [140]:
X_test.shape,y_test.shape

(torch.Size([1, 159, 32]), (159, 1))

In [141]:
# 创建一个简单的LSTM模型
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out)
        return out

# 设置超参数
input_size = 32# 输入特征的大小
hidden_size = 64 # 隐藏层的大小
num_layers = 2 # LSTM层数
output_size = 1 # 输出大小，二分类问题

# 创建模型实例
model = LSTMModel(input_size, hidden_size, num_layers, output_size)

# 定义损失函数和优化器
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


# 训练模型
num_epochs = 150
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 1 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/150], Loss: 0.6964
Epoch [2/150], Loss: 0.6853
Epoch [3/150], Loss: 0.6696
Epoch [4/150], Loss: 0.6552
Epoch [5/150], Loss: 0.6439
Epoch [6/150], Loss: 0.6385
Epoch [7/150], Loss: 0.6351
Epoch [8/150], Loss: 0.6337
Epoch [9/150], Loss: 0.6335
Epoch [10/150], Loss: 0.6324
Epoch [11/150], Loss: 0.6311
Epoch [12/150], Loss: 0.6301
Epoch [13/150], Loss: 0.6291
Epoch [14/150], Loss: 0.6286
Epoch [15/150], Loss: 0.6278
Epoch [16/150], Loss: 0.6280
Epoch [17/150], Loss: 0.6266
Epoch [18/150], Loss: 0.6262
Epoch [19/150], Loss: 0.6254
Epoch [20/150], Loss: 0.6243
Epoch [21/150], Loss: 0.6240
Epoch [22/150], Loss: 0.6222
Epoch [23/150], Loss: 0.6215
Epoch [24/150], Loss: 0.6199
Epoch [25/150], Loss: 0.6184
Epoch [26/150], Loss: 0.6186
Epoch [27/150], Loss: 0.6177
Epoch [28/150], Loss: 0.6144
Epoch [29/150], Loss: 0.6165
Epoch [30/150], Loss: 0.6143
Epoch [31/150], Loss: 0.6110
Epoch [32/150], Loss: 0.6131
Epoch [33/150], Loss: 0.6109
Epoch [34/150], Loss: 0.6089
Epoch [35/150], Loss: 0

In [142]:
# Evaluate model
model.eval()
with torch.no_grad():
    y_pred= model(X_train)
    y_pred = (y_pred.squeeze().detach().numpy() > 0.5).astype(int)

# Calculate accuracy
accuracy = np.mean(y_pred == y_train.squeeze().detach().numpy())
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.97


In [143]:
model.eval()
with torch.no_grad():
    y_pred_t= model(X_test)
    y_pred_t = (y_pred_t.squeeze().detach().numpy() > 0.5).astype(int)
accuracy_score(y_pred_t,y_test)

0.7987421383647799